In [1]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy)

In [6]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df

In [2]:
pd.set_option('display.max_columns', 50)

In [4]:
db.list_collection_names()

['games']

In [19]:
games = db.games.find()

In [20]:
final_df = create_master_df(games)

/Users/david/galvanize/super_liga_xg/mongo_to_db.py:69: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  master_df = pd.concat([attach_to_df, df], axis=0, ignore_index=True)
/Users/david/galvanize/super_liga_xg/dataframe_cleaner.py:133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  input_df['is_goal'] = input_df.loc[:, 'shot_type'].isin([9, 11, 12, 13]).astype(int)


# EDA

In [21]:
final_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
1,448548,60730.0,21.33,12.00,4.24,-0.91,-1,0.73,22336386,35,20,9.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,21.747333,11.242763,0,0,0.0
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
3,448548,185745.0,8.44,0.89,14.53,-6.66,-1,-1.00,22336723,33,20,38.883333,177285.0,22.67,6.22,3.93,19.67,-1.0,-1.0,0.0,16.803407,59.849102,1,0,0.0
4,448548,77892.0,22.22,0.44,13.92,-8.48,-1,-1.00,22336739,33,13,40.083333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,26.220122,32.065607,0,0,0.0


In [22]:
final_df.describe()

,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z2,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,is_goal
count,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,2955.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.000000,1221.0,1221.0,2945.000000,2955.000000,2955.000000,2955.000000
mean,83417.628088,15.376156,2.049354,0.202876,0.057773,-0.252988,50.018252,85681.678133,20.738591,17.941474,1.218714,0.186478,-1.0,-1.0,0.086927,17.779827,0.275928,0.109983
std,64229.759131,6.624601,3.816875,8.803820,5.448328,0.738580,26.590282,62086.040107,16.754803,11.353598,18.873264,10.908477,0.0,0.0,0.281776,6.458381,31.213869,0.312922
min,51.000000,0.000000,0.000000,-29.060000,-24.520000,-1.000000,0.250000,51.000000,0.000000,0.440000,-30.270000,-28.450000,-1.0,-1.0,0.000000,0.300000,-90.000000,0.000000
25%,30012.000000,10.670000,0.440000,-6.050000,-3.030000,-1.000000,27.616667,37152.000000,6.220000,10.670000,-14.230000,-6.960000,-1.0,-1.0,0.000000,12.441808,-22.223473,0.000000
50%,69857.000000,13.330000,0.890000,0.000000,0.300000,0.100000,50.600000,70512.000000,20.000000,14.220000,1.210000,-0.300000,-1.0,-1.0,0.000000,16.716028,0.000000,0.000000
75%,126644.000000,20.890000,2.220000,6.660000,3.030000,0.380000,73.208333,122008.000000,30.220000,23.560000,18.160000,6.960000,-1.0,-1.0,0.000000,23.315904,22.826916,0.000000
max,220396.000000,40.890000,88.440000,27.540000,26.030000,1.290000,98.850000,220396.000000,88.890000,83.560000,30.270000,28.760000,-1.0,-1.0,1.000000,47.678323,88.149038,1.000000


In [31]:
#goals that were assisted
final_df[(final_df['is_goal'] == 1) & (final_df['assisted_shot'] == 1)].head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
0,448548,99779.0,6.67,0.00,1.51,-1.21,-1,0.83,22336297,11,20,1.033333,177285.0,0.89,12.44,29.36,-4.84,-1.0,-1.0,1.0,6.838786,12.755990,1,0,1.0
15,448543,8907.0,11.56,0.44,-10.59,2.42,-1,0.08,22337581,9,136,10.916667,68605.0,21.78,15.11,3.03,-9.38,-1.0,-1.0,0.0,15.677426,-42.492487,1,0,1.0
95,448536,111421.0,18.67,0.44,-8.78,-2.12,-1,0.22,22372861,9,5,8.100000,618.0,1.78,11.56,29.66,-3.63,-1.0,-1.0,1.0,20.631464,-25.186343,1,0,1.0
152,448538,114108.0,8.89,0.00,-4.24,-1.21,-1,0.91,22383020,9,129,79.300000,193188.0,8.89,9.33,-11.80,-3.93,-1.0,-1.0,0.0,9.849350,-25.498359,1,0,1.0
153,448538,114108.0,9.33,0.00,3.03,1.21,-1,0.48,22383255,11,129,82.983333,172760.0,25.33,8.44,-27.24,7.26,-1.0,-1.0,0.0,9.809679,17.991663,1,0,1.0


In [32]:
#missed penalty kicks
final_df[final_df['shot_type'] == 44]

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
170,448541,77919.0,11.56,0.44,-0.30,-2.12,-1,1.10,22392267,44,17,15.800000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,-1.486581,0,1,0.0
332,448561,46612.0,11.56,1.78,-0.61,6.36,-1,0.29,22462959,44,869,45.666667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.576083,-3.020592,0,1,0.0


In [35]:
#converted penalty kicks
final_df[final_df['shot_type'] == 13].head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
2,448548,177285.0,12.00,1.78,0.00,-3.03,-1,0.21,22336583,13,20,26.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.000000,0.000000,0,1,1.0
142,448546,58422.0,11.56,0.44,0.30,0.61,-1,0.26,22378506,13,132,94.766667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,1.486581,0,1,1.0
143,448538,20195.0,11.56,0.00,0.30,-3.03,-1,0.10,22379135,13,12,5.900000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11.563892,1.486581,0,1,1.0
223,448537,55267.0,12.00,0.00,-0.30,-3.93,-1,0.41,23680859,13,10,78.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.003749,-1.432096,0,1,1.0
278,448556,163041.0,12.00,1.78,-0.91,-3.93,-1,0.22,22437147,13,129,83.583333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.034455,-4.336630,0,1,1.0


# Random Forest

In [65]:
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier

In [66]:
rf_columns = ['player_id', 'shot_distance', 'shot_angle', 'assisted_shot', 'is_penalty_attempt']

In [67]:
hold_off = final_df.sample(frac=.2, random_state=8)
hold_off_x = hold_off[rf_columns]
hold_off_y = hold_off['is_goal']

In [68]:
train = final_df.drop(hold_off.index).copy()
data = train[rf_columns]
y = train['is_goal']
indices = train.index.values 

In [69]:
X_train, X_test,indices_train,indices_test = train_test_split(data,indices, test_size=0.33, random_state=8)

y_train, y_test = y[indices_train],  y[indices_test]

In [70]:
random_forest_model = RandomForestClassifier(n_estimators=300, max_depth=3, verbose=1)
random_forest_model.fit(X_train, y_train)
p_random_forest = random_forest_model.predict_proba(X_test)
random_forest_ll = log_loss(y_test, p_random_forest )

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:    0.0s finished


In [71]:
p_random_forest

array([[0.89424521, 0.10575479],
       [0.90634936, 0.09365064],
       [0.93825623, 0.06174377],
       ...,
       [0.92900448, 0.07099552],
       [0.8855852 , 0.1144148 ],
       [0.45570813, 0.54429187]])

In [76]:
df_test = pd.DataFrame(X_test)
df_test['is_goal'] = y_test
df_test['prob_0'] = p_random_forest[:,0] 
df_test['prob_1'] = p_random_forest[:,1]

In [82]:
print(df_test[df_test['is_goal'] == 1]['prob_1'].sum())
df_test[df_test['is_goal'] == 1].head()

18.266360990555988


,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal,prob_0,prob_1
2794,15113.0,12.015494,2.910031,0,1,1.0,0.293490,0.706510
509,114108.0,11.676344,-40.415011,0,0,1.0,0.907008,0.092992
2273,85272.0,17.180736,-47.712851,1,0,1.0,0.926294,0.073706
756,185878.0,14.335997,7.293562,0,0,1.0,0.936567,0.063433
18,55210.0,5.037480,-13.898315,0,0,1.0,0.407489,0.592511


In [83]:
print(df_test[df_test['is_goal'] == 0]['prob_1'].sum())
df_test[df_test['is_goal'] == 0].head()

68.13319367924899


,player_id,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal,prob_0,prob_1
1733,16081.0,12.899043,37.598383,0,0,0.0,0.894245,0.105755
1290,49043.0,13.739643,-14.026129,0,0,0.0,0.906349,0.093651
334,118855.0,16.944595,-30.030079,0,0,0.0,0.938256,0.061744
1951,49584.0,15.677426,-42.492487,1,0,0.0,0.917068,0.082932
840,84739.0,14.160526,-38.318578,1,0,0.0,0.919757,0.080243


# Combining output with original dataframe

In [89]:
to_add_indices = X_test.index

In [91]:
combined_df = final_df.loc[to_add_indices, :].copy()

In [92]:
combined_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal
1733,448637,16081.0,10.22,0.44,7.87,-0.30,-1,-1.00,23202073,33,815,32.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.899043,37.598383,0,0,0.0
1290,448611,49043.0,13.33,12.89,-3.33,-3.03,-1,0.12,22965295,35,2,76.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.739643,-14.026129,0,0,0.0
334,448561,118855.0,14.67,0.44,-8.48,-2.12,-1,-1.00,22463506,33,134,52.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.944595,-30.030079,0,0,0.0
1951,448642,49584.0,11.56,2.22,-10.59,-2.12,-1,0.16,23366833,35,815,38.783333,4818.0,57.33,46.22,15.13,8.17,-1.0,-1.0,0.0,15.677426,-42.492487,1,0,0.0
840,448583,84739.0,11.11,1.78,-8.78,0.61,-1,0.63,22682090,35,10,12.166667,58112.0,4.00,12.89,23.31,-3.03,-1.0,-1.0,0.0,14.160526,-38.318578,1,0,0.0


In [93]:
combined_df['xG'] = p_random_forest[:,1]

In [95]:
combined_df.head()

,game_id,player_id,shot_coord_x1,shot_coord_x2,shot_coord_y1,shot_coord_y2,shot_coord_z1,shot_coord_z2,shot_id,shot_type,team_id,time_of_event(min),passed_from_id,pass_coord_x1,pass_coord_x2,pass_coord_y1,pass_coord_y2,pass_coord_z1,pass_coord_z2,corner_kick,shot_distance,shot_angle,assisted_shot,is_penalty_attempt,is_goal,xG
1733,448637,16081.0,10.22,0.44,7.87,-0.30,-1,-1.00,23202073,33,815,32.683333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,12.899043,37.598383,0,0,0.0,0.105755
1290,448611,49043.0,13.33,12.89,-3.33,-3.03,-1,0.12,22965295,35,2,76.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,13.739643,-14.026129,0,0,0.0,0.093651
334,448561,118855.0,14.67,0.44,-8.48,-2.12,-1,-1.00,22463506,33,134,52.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,16.944595,-30.030079,0,0,0.0,0.061744
1951,448642,49584.0,11.56,2.22,-10.59,-2.12,-1,0.16,23366833,35,815,38.783333,4818.0,57.33,46.22,15.13,8.17,-1.0,-1.0,0.0,15.677426,-42.492487,1,0,0.0,0.082932
840,448583,84739.0,11.11,1.78,-8.78,0.61,-1,0.63,22682090,35,10,12.166667,58112.0,4.00,12.89,23.31,-3.03,-1.0,-1.0,0.0,14.160526,-38.318578,1,0,0.0,0.080243


In [102]:
combined_df['xA'] = combined_df['assisted_shot'] * combined_df['xG']

In [108]:
com_col = ['game_id', 'team_id', 'player_id', 'xG', 'xA']

In [110]:
expected_df = combined_df[com_col].copy()

In [118]:
expected_df.head()

,game_id,team_id,player_id,xG,xA
1733,448637,815,16081.0,0.105755,0
1290,448611,2,49043.0,0.093651,0
334,448561,134,118855.0,0.061744,0
1951,448642,815,49584.0,0.082932,0.0829317
840,448583,10,84739.0,0.080243,0.080243


In [120]:
expected_df[expected_df['player_id'] == 16081.0]['xG'].sum()

1.5566286762011328

In [117]:
unique_players = expected_df['player_id'].unique()

In [121]:
contributions = []
for player in unique_players:
    xgsum = expected_df[expected_df['player_id'] == player]['xG'].sum()
    xasum = expected_df[expected_df['player_id'] == player]['xA'].sum()
    contributions.append([player, xgsum, xasum])

In [122]:
contributions

[[16081.0, 1.5566286762011328, 0.3769706117939714],
 [49043.0, 0.40178967673033583, 0.0],
 [118855.0, 0.38357566205625493, 0.171718447898093],
 [49584.0, 0.22578459355507507, 0.22578459355507507],
 [84739.0, 0.3245320252139665, 0.08024302203299225],
 [8513.0, 0.19199271337192592, 0.10917036143577685],
 [42505.0, 0.3123738619099341, 0.16049979490020658],
 [24111.0, 0.27649530746777984, 0.1908879186170071],
 [117606.0, 0.4031707067888287, 0.06745053824221658],
 [150346.0, 0.18244606679020992, 0.0],
 [15113.0, 1.3829726099885056, 0.18100067454113444],
 [167990.0, 0.6274211854700236, 0.6274211854700236],
 [147570.0, 0.06488304323478769, 0.0],
 [78833.0, 0.2281727899551591, 0.09740413932096241],
 [66596.0, 0.1491716463440515, 0.08674573729650221],
 [114108.0, 0.20679700759926398, 0.057227202534367215],
 [193955.0, 0.2642869401602041, 0.06450683809132994],
 [65598.0, 0.37909758153800943, 0.10767106760848097],
 [16283.0, 0.08290409883726665, 0.0],
 [36117.0, 0.24160395641766225, 0.16132112034

In [125]:
 = sorted(a, key=lambda x: x.modified, reverse=True)

In [126]:
contributions

[[9146.0, 1.6817292742256678, 1.1667440755211853],
 [16081.0, 1.5566286762011328, 0.3769706117939714],
 [20195.0, 1.4732467245386789, 0.21483084237431413],
 [15113.0, 1.3829726099885056, 0.18100067454113444],
 [24256.0, 1.3742625666044652, 1.003555660664969],
 [135386.0, 1.291808441338778, 0.0958017839700637],
 [203.0, 1.2283728945240258, 0.7210372348236189],
 [488.0, 1.196300782810391, 0.07851492385482065],
 [17266.0, 1.1235661411293063, 0.11603612371331425],
 [17217.0, 1.100375879215592, 0.18284571689746815],
 [17135.0, 1.0938719091200066, 0.8628765070941985],
 [101.0, 1.0934074753530603, 0.08144762773950953],
 [85468.0, 1.0591335744041521, 0.5541629744641565],
 [98292.0, 0.9785676239840169, 0.720953832735686],
 [39414.0, 0.9713067461594298, 0.8488710071015824],
 [52390.0, 0.9681956469602371, 0.9681956469602371],
 [172565.0, 0.9447697176547198, 0.5807515225373133],
 [618.0, 0.9338287425476917, 0.0],
 [63200.0, 0.9011840206760104, 0.6429549064828591],
 [55502.0, 0.8886281099760432, 0.